In [ ]:
import kagglehub

path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Path to dataset files: /kaggle/input/brazilian-ecommerce


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib.gridspec import GridSpec
pd.set_option('display.max_columns', 100)
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go
import json
import requests
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap, HeatMapWithTime, LocateControl
from wordcloud import WordCloud
from collections import Counter
from PIL import Image

import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import joblib

In [ ]:
raw_path = '/kaggle/input/brazilian-ecommerce/'
olist_customer = pd.read_csv(raw_path + 'olist_customers_dataset.csv')
olist_geolocation = pd.read_csv(raw_path + 'olist_geolocation_dataset.csv')
olist_orders = pd.read_csv(raw_path + 'olist_orders_dataset.csv')
olist_order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')
olist_order_payments = pd.read_csv(raw_path + 'olist_order_payments_dataset.csv')
olist_order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
olist_products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
olist_sellers = pd.read_csv(raw_path + 'olist_sellers_dataset.csv')

In [ ]:
datasets = {
    "customers": olist_customer,
    "geolocation": olist_geolocation,
    "orders": olist_orders,
    "order_items": olist_order_items,
    "payments": olist_order_payments,
    "reviews": olist_order_reviews,
    "products": olist_products,
    "sellers": olist_sellers
}

for name, df in datasets.items():
    print(f"\n===== {name.upper()} =====")
    print(df.shape)
    print(df.head())
    print(df.info())



===== CUSTOMERS =====
(99441, 5)
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column

In [ ]:
for name, df in datasets.items():
    print(f"\n--- Missing values in {name} ---")
    print(df.isna().sum())


--- Missing values in customers ---
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

--- Missing values in geolocation ---
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

--- Missing values in orders ---
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

--- Missing values in order_items ---
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

--- 

In [ ]:
for df in [olist_customer, olist_geolocation, olist_orders,
           olist_order_items, olist_order_payments, olist_order_reviews,
           olist_products, olist_sellers]:
    df.drop_duplicates(inplace=True)

In [ ]:
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
olist_orders[date_cols] = olist_orders[date_cols].apply(pd.to_datetime)

In [ ]:
olist_orders = olist_orders[olist_orders["order_status"] == "delivered"]
olist_orders = olist_orders.dropna(subset=["order_delivered_customer_date"])

In [ ]:
olist_order_items['shipping_limit_date'] = pd.to_datetime(olist_order_items['shipping_limit_date'])

In [ ]:
olist_order_reviews["review_creation_date"] = pd.to_datetime(olist_order_reviews["review_creation_date"])
olist_order_reviews["review_answer_timestamp"] = pd.to_datetime(olist_order_reviews["review_answer_timestamp"])
olist_order_reviews["review_comment_title"] = olist_order_reviews["review_comment_title"].fillna("")
olist_order_reviews["review_comment_message"] = olist_order_reviews["review_comment_message"].fillna("")

In [ ]:
olist_products["product_category_name"] = (
    olist_products["product_category_name"].fillna("unknown")
)
cols_num = [
    "product_name_lenght", "product_description_lenght", "product_photos_qty",
    "product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"
]

for col in cols_num:
    olist_products[col] = (
        olist_products.groupby("product_category_name")[col]
        .transform(lambda x: x.fillna(x.median()))
    )

In [ ]:
olist_customer["customer_zip_code_prefix"] = olist_customer["customer_zip_code_prefix"].astype(str)
olist_geolocation["geolocation_zip_code_prefix"] = olist_geolocation["geolocation_zip_code_prefix"].astype(str)
olist_sellers["seller_zip_code_prefix"] = olist_sellers["seller_zip_code_prefix"].astype(str)

In [ ]:
valid_ids = set(olist_order_items["order_id"])
olist_orders = olist_orders[olist_orders["order_id"].isin(valid_ids)]

In [ ]:
payments_agg = olist_order_payments.groupby("order_id").agg({
    "payment_value": "sum",
    "payment_installments": "max",
    "payment_type": lambda x: x.iloc[0]  # thường chỉ 1 loại
}).reset_index()

In [ ]:
olist_orders["delivery_time"] = (
    olist_orders["order_delivered_customer_date"] -
    olist_orders["order_purchase_timestamp"]
).dt.days

olist_orders["delay_vs_estimated"] = (
    olist_orders["order_delivered_customer_date"] -
    olist_orders["order_estimated_delivery_date"]
).dt.days

In [ ]:
fact = (
    olist_order_items
    .merge(olist_orders, on="order_id", how="left")
    .merge(olist_customer, on="customer_id", how="left")
    .merge(olist_products, on="product_id", how="left")
    .merge(olist_sellers, on="seller_id", how="left")
    .merge(payments_agg, on="order_id", how="left")
    .merge(olist_order_reviews, on="order_id", how="left")
)


In [ ]:
fact["purchase_date"] = fact["order_purchase_timestamp"].dt.date
fact["purchase_year"] = fact["order_purchase_timestamp"].dt.year
fact["purchase_month"] = fact["order_purchase_timestamp"].dt.to_period("M")
fact["purchase_weekday"] = fact["order_purchase_timestamp"].dt.weekday
fact["delivery_time"] = (
    fact["order_delivered_customer_date"] - fact["order_purchase_timestamp"]
).dt.days

fact["estimated_delivery_time"] = (
    fact["order_estimated_delivery_date"] - fact["order_purchase_timestamp"]
).dt.days

fact["delivery_delay"] = (
    fact["order_delivered_customer_date"] - fact["order_estimated_delivery_date"]
).dt.days
fact["item_total"] = fact["price"] + fact["freight_value"]
fact["has_comment"] = (fact["review_comment_message"].str.len() > 0).astype(int)


In [ ]:
fact.info()
fact.to_csv('fact.csv', index=False)
fact.to_parquet('fact.parquet', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113314 entries, 0 to 113313
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       113314 non-null  object        
 1   order_item_id                  113314 non-null  int64         
 2   product_id                     113314 non-null  object        
 3   seller_id                      113314 non-null  object        
 4   shipping_limit_date            113314 non-null  datetime64[ns]
 5   price                          113314 non-null  float64       
 6   freight_value                  113314 non-null  float64       
 7   customer_id                    110832 non-null  object        
 8   order_status                   110832 non-null  object        
 9   order_purchase_timestamp       110832 non-null  datetime64[ns]
 10  order_approved_at              110817 non-null  datetime64[ns]
 11  

In [ ]:
# fact.to_csv('fact.csv', index=False)

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

Authenticated


In [ ]:
PROJECT_ID = "nimble-climber-478516-q6"
BUCKET_NAME = "bdabi-group7"
DESTINATION_BLOB_NAME = "preprocessed/preprocessed.parquet"
LOCAL_FILE_PATH = "/content/fact.parquet"

In [ ]:
!pip install --quiet google-cloud-storage

from google.cloud import storage

client = storage.Client(project=PROJECT_ID)
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(DESTINATION_BLOB_NAME)

blob.upload_from_filename(LOCAL_FILE_PATH)

In [ ]:
print(f"Uploaded to gs://{BUCKET_NAME}/{DESTINATION_BLOB_NAME}")
print("Using the url above to access the resource for processing")

Uploaded to gs://bdabi-group7/preprocessed/preprocessed.parquet
Using the url above to access the resource for processing
